In [2]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Path to Excel file and image folder
data_path = 'C:\\Users\\hasan\\OneDrive\\Desktop\\jupyter\\Automated_report\\DATASET.xlsx'
image_base_path = 'C:\\Users\\hasan\\OneDrive\\Desktop\\jupyter\\Automated_report\\MATERIALS'

# Load the dataset
df = pd.read_excel(data_path)

# Checking the structure of the dataset
print(df.head())

# Encode categorical variables like 'Material'
le_material = LabelEncoder()
df['Material_Encoded'] = le_material.fit_transform(df['Material'])

# Clean the 'Force (N)' column: Convert to numeric, replace errors with NaN
df['Force (N)'] = pd.to_numeric(df['Force (N)'], errors='coerce')

# Generate image filenames using Force (N) values (assuming naming convention)
def generate_image_path(row):
    # Handle NaN by filling with a default value (e.g., 0)
    force = 0 if pd.isna(row['Force (N)']) else int(row['Force (N)'])  # Fill NaN with 0
    # Create filename based on force
    image_filename = f"D_{force}.png"
    # Return full path to the image
    return os.path.join(image_base_path, image_filename)

# Apply the function to generate the image paths
df['Image_Path'] = df.apply(generate_image_path, axis=1)

# Function to load images and resize them to a consistent size
def load_image(img_path):
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (128, 128))  # Resize to 128x128
        return img
    return np.zeros((128, 128, 3))  # Fallback if image not found

# Load images based on the paths in the dataset
df['Image'] = df['Image_Path'].apply(lambda x: load_image(x))

# Convert the image column to a numpy array
X = np.array(list(df['Image']))  # Images
Y = df[['Max Stress(Mpa)', 'Min Stress(Mpa)', 'Max def(mm)', 'youngs modulus(pa)', 'poisions raito', 'Thickness(mm)', 'Diameter(mm)', 'Force (N)']].values  # Target variables

# Save the preprocessed data for model training
np.save('X_images.npy', X)
np.save('Y_labels.npy', Y)

print("Data preprocessing complete. Saved X_images.npy and Y_labels.npy.")


           Material  Density(kg/m3)  youngs modulus(pa)  poisions raito  \
0  Structural steel          7850.0        2.000000e+11             0.3   
1  Structural steel          7850.0        2.000000e+11             0.3   
2  Structural steel          7850.0        2.000000e+11             0.3   
3  Structural steel          7850.0        2.000000e+11             0.3   
4  Structural steel          7850.0        2.000000e+11             0.3   

   Thickness(mm)  Diameter(mm)  Force (N)  Max def(mm)  Min Stress(Mpa)  \
0            5.0          20.0    50000.0     0.057332           13.386   
1            5.0          20.0    51000.0     0.058478           13.654   
2            5.0          20.0    52000.0     0.059625           13.921   
3            5.0          20.0    53000.0     0.060771           14.189   
4            5.0          20.0    54000.0     0.061918           14.457   

   Max Stress(Mpa)  case  
0           326.84   1.0  
1           333.38   2.0  
2           339.9

In [3]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import joblib  # Import joblib to save the model

# Load the data from an Excel file
df = pd.read_excel(data_path)

# Drop rows where any of the target variables have NaN values
df_clean = df.dropna(subset=['Min Stress(Mpa)', 'Max Stress(Mpa)', 'Max def(mm)'])

# Prepare the features (X) and target variables (y)
X = df_clean[['Thickness(mm)', 'Diameter(mm)', 'Force (N)', 'Max def(mm)']]
y = df_clean[['Min Stress(Mpa)', 'Max Stress(Mpa)', 'Max def(mm)']]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Regressor with MultiOutputRegressor
rf_model = RandomForestRegressor()
rf_multi_model = MultiOutputRegressor(rf_model)

# Parameter grid for Random Forest
# Expanded parameter grid for Random Forest
rf_param_grid = {
    'estimator__n_estimators': [100, 200, 300],
    'estimator__max_depth': [None, 10, 20, 30],
    'estimator__max_features': ['sqrt', 'log2'],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4],
}


rf_grid_search = GridSearchCV(estimator=rf_multi_model, param_grid=rf_param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
rf_grid_search.fit(X_train, y_train)

# Best Random Forest model
best_rf_model = rf_grid_search.best_estimator_

# Predictions and evaluation for Random Forest
y_train_pred_rf = best_rf_model.predict(X_train)
y_test_pred_rf = best_rf_model.predict(X_test)

train_mse_rf = mean_squared_error(y_train, y_train_pred_rf)
test_mse_rf = mean_squared_error(y_test, y_test_pred_rf)

print("Random Forest Regressor")
print("Training MSE:", train_mse_rf)
print("Testing MSE:", test_mse_rf)

# Save the trained Random Forest model
model_filename = 'best_random_forest_model.pkl'
joblib.dump(best_rf_model, model_filename)
print(f'Model saved as {model_filename}')


Fitting 3 folds for each of 216 candidates, totalling 648 fits
Random Forest Regressor
Training MSE: 681774.9199333246
Testing MSE: 5406.1243308257535
Model saved as best_random_forest_model.pkl


In [12]:
pip install xgboost


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 1.3/124.9 MB 8.4 MB/s eta 0:00:15
    --------------------------------------- 2.1/124.9 MB 7.3 MB/s eta 0:00:17
   - -------------------------------------- 3.1/124.9 MB 5.3 MB/s eta 0:00:24
   - -------------------------------------- 4.2/124.9 MB 5.4 MB/s eta 0:00:23
   - -------------------------------------- 5.5/124.9 MB 5.4 MB/s eta 0:00:23
   -- ------------------------------------- 6.3/124.9 MB 5.1 MB/s eta 0:00:24
   -- ------------------------------------- 7.9/124.9 MB 5.5 MB/s eta 0:00:22
   --- ------------------------------------ 9.4/124.9 MB 5.7 MB/s eta 0:00:21
   --- ------------------------------------ 10.5/124.9 MB 5.6 MB/s eta 0:00:21
   --- ------------------------------------ 11.0/124.9 MB 5.5 MB/s eta 0:00:21
   --- ------------------------------------ 12.3/124.9 MB 5.4 MB/s et

In [18]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
import joblib  # Import joblib to save the model
from sklearn.metrics import mean_squared_error

xgb_model = XGBRegressor()

# Multi-output regressor with XGBoost
xgb_multi_model = MultiOutputRegressor(xgb_model)

xgb_param_grid = {
    'estimator__n_estimators': [100, 200],
    'estimator__learning_rate': [0.01, 0.1],
    'estimator__max_depth': [3, 5, 7],
}

xgb_grid_search = GridSearchCV(estimator=xgb_multi_model, param_grid=xgb_param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
xgb_grid_search.fit(X_train, y_train)

# Best XGBoost model
best_xgb_model = xgb_grid_search.best_estimator_



# Predictions and evaluation for XGBoost
y_train_pred_xgb = best_xgb_model.predict(X_train)
y_test_pred_xgb = best_xgb_model.predict(X_test)

# Calculate the MSE for training and testing sets
train_mse_xgb = mean_squared_error(y_train, y_train_pred_xgb)
test_mse_xgb = mean_squared_error(y_test, y_test_pred_xgb)

# Print the MSE values
print("XGBoost Regressor")
print("Training MSE:", train_mse_xgb)
print("Testing MSE:", test_mse_xgb)

# Save the trained XGBoost model
xgb_model_filename = 'best_xgboost_model.pkl'
joblib.dump(best_xgb_model, xgb_model_filename)
print(f'XGBoost model saved as {xgb_model_filename}')



Fitting 3 folds for each of 12 candidates, totalling 36 fits
XGBoost Regressor
Training MSE: 0.6094946177270267
Testing MSE: 3.9261137962038384
XGBoost model saved as best_xgboost_model.pkl


In [19]:
pip install python-pptx


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [30]:
#ppt code
import numpy as np
import cv2
import joblib
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
import pandas as pd

# Load the pre-trained model
model = joblib.load('best_xgboost_model.pkl')

# Function to preprocess the image
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (128, 128))  # Resize to 128x128
        img_flattened = img.flatten()
        return img_flattened
    return np.zeros((128 * 128 * 3))  # Fallback if image not found

# Function to predict features
def predict_features(thickness, diameter, force, img_path):
    # Prepare input features (adjusted to match model's expected input size)
    input_data = np.array([[thickness, diameter, force, 0]])  # Assuming the model expects 4 features
    
    # Make prediction
    predicted_features = model.predict(input_data)
    
    # Preprocess the image
    img = preprocess_image(img_path)
    
    return predicted_features, img

# Function to create a PowerPoint report
def create_report(image_path, predicted_features, thickness, diameter, force, material):
    prs = Presentation()
    slide_layout = prs.slide_layouts[5]  # Blank slide layout
    slide = prs.slides.add_slide(slide_layout)

    # Add title to the slide
    title = slide.shapes.title
    title.text = "Stress and Deformation Report"
    title.text_frame.paragraphs[0].font.size = Pt(24)  # Set title font size

    # Add image to slide
    slide.shapes.add_picture(image_path, Inches(0.5), Inches(1.5), width=Inches(4.5), height=Inches(3))

    # Add a table to the slide
    rows, cols = 8, 2  # Adjusted the number of rows to accommodate input and predicted features
    left = Inches(5)
    top = Inches(1.5)
    width = Inches(4)
    height = Inches(3)
    
    table = slide.shapes.add_table(rows, cols, left, top, width, height).table

    # Set column widths
    table.columns[0].width = Inches(2)
    table.columns[1].width = Inches(2)

    # Set table header
    table.cell(0, 0).text = "Features"
    table.cell(0, 1).text = "Values"
    for cell in table.rows[0].cells:
        cell.fill.solid()
        cell.fill.fore_color.rgb = RGBColor(220, 220, 220)  # Header background color
        cell.text_frame.paragraphs[0].font.bold = True
        cell.text_frame.paragraphs[0].font.size = Pt(30)  # Increased font size for header
        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    # Add input features to the table
    input_data = [
        ("Thickness (mm)", thickness),
        ("Diameter (mm)", diameter),
        ("Force (N)", force),
        ("Material", material),
        ("", ""),  # Empty row for spacing
        ("Predicted Features", "")
    ]

    for i, (feature, value) in enumerate(input_data):
        table.cell(i + 1, 0).text = feature
        table.cell(i + 1, 1).text = str(value)

    # Add predicted features to the table
    max_stress, min_stress, max_deformation = predicted_features[0]
    
    predicted_data = [
        ("Max Stress (MPa)", max_stress),
        ("Min Stress (MPa)", min_stress),
        ("Max Deformation (mm)", max_deformation)
    ]

    for i, (feature, value) in enumerate(predicted_data):
        table.cell(i + 5, 0).text = feature
        table.cell(i + 5, 1).text = f"{value:.2f}"  # Format to 2 decimal places

    # Customize font size for all cells
    for row in table.rows:
        for cell in row.cells:
            cell.text_frame.paragraphs[0].font.size = Pt(12)  # Set font size for all cells

    # Save the presentation
    report_path = 'stress_deformation_report.pptx'
    prs.save(report_path)
    print(f'Report saved at {report_path}')

# Example usage
thickness = 5  # Input thickness
diameter = 20  # Input diameter
force = 51000  # Input force
material = "Aluminum"  # Input material
image_path = 'C:\\Users\\hasan\\OneDrive\\Desktop\\jupyter\\Automated_report\\MATERIALS\\AA_T5_D20\\D_51000.png'  # Path to the image

# Call predict_features with the correct number of arguments
predicted_features, img = predict_features(thickness, diameter, force, image_path)
create_report(image_path, predicted_features, thickness, diameter, force, material)

Report saved at stress_deformation_report.pptx


In [ ]:
if __name__ == '__main__':
    app.run(debug=True)